In [3]:
!pip install -U scikit-learn
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn in /home/users/rvignav/.local/lib/python3.6/site-packages (0.24.1)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached scipy-1.5.4-cp36-cp36m-manylinux1_x86_64.whl (25.9 MB)
You should consider upgrading via the '/share/software/user/open/python/3.6.1/bin/python3.6 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/software/user/open/python/3.6.1/bin/python3.6 -m pip install --upgrade pip' command.


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import models
from torchvision import transforms
from tqdm import tqdm
import pandas as pd
import numpy as np
import cv2
import time
from PIL import Image
import glob
from sklearn.metrics import classification_report, roc_auc_score

ImportError: numpy.core.multiarray failed to import

In [ ]:
def check_cuda():
    is_cuda = False
    if torch.cuda.is_available():
        is_cuda = True
    return is_cuda

class DRDataset(Dataset):
    def __init__(self, csv_path):
        self.to_tensor = transforms.ToTensor()
        self.data_info = pd.read_csv(csv_path, header=None)
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        single_image_name = '/scratch/groups/rubin/rvignav/input/train/' + self.image_arr[index] + '.jpeg'
        a = cv2.imread(single_image_name)
        
        scale = 300
        a = cv2.addWeighted(a, 4, cv2.GaussianBlur(a, (0,0), scale/30), -4, 128)
        b = np.zeros(a.shape)
        cv2.circle(b, (int(a.shape[1]/2), int(a.shape[0]/2)), int(scale*0.9), (1,1,1), -1, 8, 0)
        a = a*b + 128*(1-b)
        a = cv2.resize(a, (256,256))
        
        img_as_tensor = self.to_tensor(a)

        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len
    
full_dataset = DRDataset('/home/users/rvignav/trainLabels.csv') #'/scratch/groups/rubin/rvignav/input/train/trainLabels.csv'

train_size = int(0.9 * len(full_dataset)) 
val_size = int(0.05 * len(full_dataset)) 
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, 
                                                                         [train_size, val_size, test_size])

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                          shuffle=True, num_workers=4, pin_memory=True)

valid_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                          shuffle=True, num_workers=4, pin_memory=True)

test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
                                          shuffle=True)

In [ ]:
model = torch.load('/scratch/groups/rubin/adam_balanced_not_stylized_5epochs') #adam_not_stylized_2epochs

In [ ]:
y_true = []
y_pred = []
y_tot = []

classes = ['none', 'mild', 'moderate', 'severe', 'proliferative']
with torch.no_grad():
    for data in test_data_loader:
        images, labels = data
        images = np.array(images, int) 
        images = torch.Tensor(images).float()
        labels = np.array(labels, int) 
        labels = torch.Tensor(labels).long()
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(len(labels)):
            y_true.append(int(labels[i].item()))
            y_pred.append(int(predicted[i].item()))
            y_tot.append(outputs[i])

#measure sensitivity for each class
#measure AUC (AUC: 0.75+ is our target) & ROC (for the all data and for the balanced dataset)

In [ ]:
print(classification_report(y_true, y_pred, target_names=classes))

In [ ]:
# print(roc_auc_score(y_true, y_tot, average='weighted', multi_class='ovr'))